In [1]:
!pip3 install nlp
!pip3 install ipywidgets
!pip3 install torch
!pip3 install transformers
!pip3 install datasets
!pip3 install tensorboard

In [2]:
from transformers import BertForSequenceClassification
from transformers import AdamW
from transformers import BertTokenizer
from transformers import Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import pandas as pd
import datasets

In [7]:
fields = ['comment_text', 'toxicity', 'target']
dataset = datasets.load_dataset('csv', data_files={'train': \
                                'data/pre_processed/subsets/25_percent/lemma/train_lemma_nopunct_cleaned_sentencized.csv',\
                                'valid': 'data/pre_processed/valid_lemma_nopunct_cleaned_sentencized.csv', \
                                'test':})

train = dataset["train"]
valid = dataset["valid"]

Using custom data configuration default-035ee9258b084f01


Dataset csv downloaded and prepared to /home/krise/.cache/huggingface/datasets/csv/default-035ee9258b084f01/0.0.0/965b6429be0fc05f975b608ce64e1fa941cc8fb4f30629b523d2390f3c0e1a93. Subsequent calls will reuse this data.


In [4]:
#model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
tokenizer = BertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch['comment_text'], padding=True, truncation=True)

train = train.map(tokenize, batched=True, batch_size=len(train))
valid = valid.map(tokenize, batched=True, batch_size=len(valid))
train.rename_column_("target", "labels")
valid.rename_column_("target", "labels")
train.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
valid.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [5]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'auc': auc
    }

training_args = TrainingArguments(
    num_train_epochs= 3,
    output_dir = "./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    tokenizer = tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train,
    eval_dataset=valid
)
trainer.train()

Step,Training Loss
500,0.296700
1000,0.188400
1500,0.195800
2000,0.170800
2500,0.157400
3000,0.166100
3500,0.173500
4000,0.182900
4500,0.135700
5000,0.145800


TrainOutput(global_step=135348, training_loss=0.13164770236430898, metrics={'train_runtime': 54870.4757, 'train_samples_per_second': 2.467, 'total_flos': 153984076063641000, 'epoch': 3.0})

In [9]:
trainer.evaluate(valid)

{'eval_loss': 0.24274355173110962,
 'eval_accuracy': 0.9423197283809756,
 'eval_f1': 0.6364969108869419,
 'eval_precision': 0.6396001403016486,
 'eval_recall': 0.6334236487425317,
 'eval_auc': 0.8012516761945855,
 'eval_runtime': 6019.2039,
 'eval_samples_per_second': 59.99,
 'epoch': 3.0}